In [24]:
from gensim.models import word2vec,Word2Vec
from gensim.utils import tokenize
import pandas as pd
import numpy as np
import pandas as pd
import re
from nltk.tokenize import sent_tokenize
from gensim.models.phrases import Phrases, Phraser
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [25]:
data = pd.read_csv("data_clean.csv")

In [26]:
data.head()

,text,label
0,alway wrote seri complet stinkfest jim belushi...,0
1,1st watch 10dirstev purcel typic mari kate ash...,0
2,movi poorli written direct fell asleep minut m...,0
3,interest thing miryang secret sunshin actor je...,1
4,first read berlin meer didnt expect much thoug...,0


In [27]:
data.label.value_counts()

1    24884
0    24698
Name: label, dtype: int64

In [28]:
data = data.dropna()

In [29]:
def preprocess_label(review):
    review_text = review
    words = review_text.split()
    return (words)
def labels_to_sentences(label):
    raw_sentences = sent_tokenize(label.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence)>0:
            sentences.append(preprocess_label(raw_sentence))
    return sentences

In [30]:
labels_to_sentences(data['text'][0])

[['alway',
  'wrote',
  'seri',
  'complet',
  'stinkfest',
  'jim',
  'belushi',
  'involv',
  'heavili',
  'one',
  'day',
  'tragic',
  'happenst',
  'occur',
  'white',
  'sox',
  'game',
  'end',
  'realiz',
  'remot',
  'way',
  'side',
  'room',
  'somehow',
  'could',
  'gotten',
  'walk',
  'across',
  'room',
  'get',
  'remot',
  'even',
  'tv',
  'turn',
  'channel',
  'get',
  'walk',
  'across',
  'countri',
  'watch',
  'tv',
  'anoth',
  'state',
  'nut',
  'said',
  'decid',
  'hang',
  'tight',
  'couch',
  'take',
  'whatev',
  'fate',
  'store',
  'fate',
  'store',
  'episod',
  'show',
  'episod',
  'rememb',
  'littl',
  'except',
  'made',
  'broad',
  'gener',
  'sweep',
  'blanket',
  'judgment',
  'base',
  'zero',
  'object',
  'experienti',
  'evid',
  'noth',
  'whatsoev',
  'back',
  'opinion',
  'complet',
  'right',
  'show',
  'total',
  'crudpi',
  'belushi',
  'comed',
  'deliveri',
  'hairi',
  'lighthous',
  'foghorn',
  'women',
  'physic',
  'att

In [31]:
sentences = []
for review in data["text"]:
    if type(review) == float:
        continue
    sentences += labels_to_sentences(review)
len(sentences)

49582

In [32]:
#sent = [row.split() for row in data['text'] if not type(row) == float]
#phrases = Phrases(sent, min_count=1, progress_per=10000)
#sentences = phrases[sent]

In [33]:
num_features = 300  
min_word_count = 3
num_workers = 4     
context = 10       
downsampling = 1e-3 # (0.001) 

from gensim.models import word2vec
model = word2vec.Word2Vec(sentences,\
                          workers=num_workers,\
                          size=num_features,\
                          min_count=min_word_count,\
                          window=context,
                          sample=downsampling,
                          sg = 1
                         )

model.init_sims(replace=True)

In [34]:
model.wv.vectors.shape

(42553, 300)

In [35]:
model.wv.most_similar("awsom")

[('greatit', 0.8054613471031189),
 ('nameand', 0.7951759099960327),
 ('ripa', 0.7915255427360535),
 ('thoe', 0.7837172746658325),
 ('appereantli', 0.7826181054115295),
 ('petsbr', 0.7802642583847046),
 ('yah', 0.7759445905685425),
 ('efron', 0.7745994329452515),
 ('funnyand', 0.7739788293838501),
 ('performedbr', 0.7736026644706726)]

In [36]:
def featureVecMethod(words, model, num_features):
    featureVec = np.zeros(num_features,dtype="float32")
    nwords = 0
    
    index2word_set = set(model.wv.index2word)
    
    for word in  words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])

    featureVec = np.divide(featureVec, nwords)
    return featureVec
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:
        reviewFeatureVecs[counter] = featureVecMethod(review, model, num_features)
        counter = counter+1
        
    return reviewFeatureVecs
data_vects = []
for review in data['text']:
        data_vects.append(preprocess_label(review))
    
dataAvgVects = getAvgFeatureVecs(data_vects, model, num_features)

//anaconda3/envs/cloudcompt/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [37]:
dataVects = []
for item in dataAvgVects:
    item[np.isnan(item)] = 0
    dataVects.append(item)

In [38]:
len(dataVects)

49582

In [39]:
len(data)

49582

In [40]:
dataVects = np.asarray(dataVects)

In [41]:
encoder = LabelBinarizer()
y = encoder.fit_transform(data['label'])

In [42]:
X_train, X_test, y_train, y_test = train_test_split(dataVects,y,test_size=0.2,random_state = 42)

In [43]:
y_test.shape

(9917, 1)

In [44]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout

In [46]:
classifier = Sequential()
classifier.add(Dense(units=128,activation='relu',input_shape=(300,)))
classifier.add(Dense(units=128,activation='relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(units=128,activation='relu'))
classifier.add(Dense(units=128,activation='relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(units=128,activation='relu'))
classifier.add(Dense(units=128,activation='relu'))
classifier.add(Dropout(0.2))
classifier.add(Dense(units=128,activation='relu'))
classifier.add(Dense(units=128,activation='relu'))

classifier.add(Dense(units=1,activation='sigmoid'))

classifier.compile(optimizer='adam',
              loss='binary_crossentropy',metrics=['accuracy'])

In [47]:
classifier.fit(x=X_train, 
          y=y_train, 
          epochs=30,
          validation_data=(X_test, y_test), verbose=1)

Train on 39665 samples, validate on 9917 samples
Epoch 1/30
39665/39665 [==============================] - 13s 330us/sample - loss: 0.3501 - accuracy: 0.8464 - val_loss: 0.3151 - val_accuracy: 0.8553
Epoch 2/30
39665/39665 [==============================] - 11s 277us/sample - loss: 0.3077 - accuracy: 0.8740 - val_loss: 0.2995 - val_accuracy: 0.8833
Epoch 3/30
39665/39665 [==============================] - 11s 284us/sample - loss: 0.2928 - accuracy: 0.8804 - val_loss: 0.2844 - val_accuracy: 0.8733
Epoch 4/30
39665/39665 [==============================] - 12s 290us/sample - loss: 0.2905 - accuracy: 0.8828 - val_loss: 0.2792 - val_accuracy: 0.8835
Epoch 5/30
39665/39665 [==============================] - 12s 292us/sample - loss: 0.2894 - accuracy: 0.8828 - val_loss: 0.2912 - val_accuracy: 0.8765
Epoch 6/30
39665/39665 [==============================] - 12s 291us/sample - loss: 0.2858 - accuracy: 0.8826 - val_loss: 0.2780 - val_accuracy: 0.8822
Epoch 7/30
39665/39665 [=====================

In [68]:
classifier.predict_classes(np.expand_dims(X_train[1],axis=0))[0][0]

1

In [62]:
y_train[1][]

array([1])

In [65]:
model.save("word2vec.model")

In [66]:
classifier.save("sentiment_classifier.h5")

In [5]:
model = Word2Vec.load("word2vec.model")

In [6]:
from tensorflow.keras.models import load_model
classifier = load_model("sentiment_classifier.h5")

In [19]:
data = getAvgFeatureVecs([data['text'][0]],model,300)

//anaconda3/envs/cloudcompt/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  # Remove the CWD from sys.path while we load stuff.


In [20]:
dataVects = []
for item in data:
    item[np.isnan(item)] = 0
    dataVects.append(item)

In [23]:
classifier.predict(dataVects[0])

ValueError: Input 0 of layer sequential_30 is incompatible with the layer: expected axis -1 of input shape to have value 300 but received input with shape [None, 1]